# 6

This part (#6) revolves all around eigenvalues, eigenvectors, and corresponding matrix decompositions.
Let's start with initialization as usual:

In [ ]:
import numpy as np                # basic arrays, vectors, matrices
import scipy as sp                # matrix linear algebra 

import matplotlib                 # plotting
import matplotlib.pyplot as plt   # plotting

%matplotlib inline

from IPython.core.display import HTML
HTML("""<style>.output_png { display: table-cell; text-align: center; vertical-align: middle; }</style>""");

<div class="alert alert-success">

**Task 0**: For more information on Numpy in general, and arrays in particular have a look at the following tutorials:

- [Numpy Tutorial for Beginners](https://www.kaggle.com/legendadnan/numpy-tutorial-for-beginners-data-science)
- [Python Numpy Tutorial with Jupyter and Colab](https://cs231n.github.io/python-numpy-tutorial/)
- [Numpy Tutorials](https://numpy.org/numpy-tutorials/)
</div>

<div class="alert alert-info">

### Power Iteration to Compute the Largest Eigenpair
</div>

In the course, we discussed briefly several methods to compute eigenvalues of a matrix $A$. Among these, the *Power Iteration* is the simplest. However, it is a good illustration of the general idea behind eigenvalue algorithms. It calculates the largest eigenvalue and the corresponding eigenvector in an iterative manner by repeated application of $A$ to a vector $\mathbf{b}$.

<div class="alert alert-success">

**Task 1**: Complete the function `power_iteration` below to implement the power iteration algorithm to compute the largest eigenpair, given the square matrix `A`. The function should have the following properties:

- It should return the sequence of $\mu_k$ and $\mathbf{b}$, where $\mu_k$ is the Rayleigh coefficient obtained in the $k$-th iteration (This is used for visual verification in the code below the function by plotting the $\mu_k$). The other output $\mathbf{b}$ is the approximation of the eigenvector.
- `power_iteration` should terminate if either the maximum number of iterations (`maxiter`) is reached, or if $(\mu_k,b_k)$ is (numerically) an eigenpair, i.e. $Ab_k$ and $\mu_k b_k$ are close. You may use [`numpy.allclose`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html) to check this.
</div>

In [ ]:
def power_iteration(A, maxiter = 50):
    """perform power iteration on A and return the sequence of Rayleigh coefficients"""
    b = np.random.random(A.shape[1])
    mu = [] # We went with the empty list instead of the list full of zeros, to be able to append new elements to it, without going out of bounds :-)
    
    for i in range(maxiter):
        b_new = A@b / np.linalg.norm(A@b) # b_new would pretty much change every time, since we use it's old value to compute the new one, after every iteration
        mu_current = (b.T@A@b) / (b.T@b) # The Rayleigh coefficient is calculated every itteration, and corresponds to the current corresponding eigenvector.
        mu.append(mu_current) # each one is added to the array
        
        if np.allclose(A @ b, mu_current * b):
            mu = np.asarray(mu) #np.asarray to make sure that we get a NumPy Array.
            return mu,b 
        b = b_new
        
    mu = np.asarray(mu)
    
    return mu,b     


# test power iteration (largest eigenvalue of A = 18)
A = np.array([[9, 0, 3], [4, 6, 12], [15, 9, 3]])

mu, b = power_iteration(A)

print( "largest eigenvalue =", mu[-1], "(%d iterations)"%len(mu) )
print( "corresponding eigenvector = ",b)
# visualize the convergence of the Rayleigh coefficients

fig = plt.plot( mu,linewidth=4.0)
plt.grid()

If the algorithm is correct, it should return the maximum eigenvalue 18 of $A$ after a few (< 20) iterations.

Let's inspect convergence by plotting $\Delta_k = |\mu_{k+1} - \mu_k|$, i.e. the order of magnitude of successive updates, in a logarithmic plot.

In [ ]:
def plot_convergence(ax, mu):
    delta = np.abs(mu[1:]-mu[:-1])
    ax.plot(delta)
    ax.set_yscale('log')
    ax.grid(True)
    
plot_convergence(plt.gca(), mu)

<div class="alert alert-success">

**Task 2**: visualize and compare the convergence for the following three matrices:

$$A_1 = \mathrm{diag}(10,2,1), \ A_2 = \mathrm{diag}(10,8,1),\ A_3 = \mathrm{diag}(10,9.9,1).$$

What is the explanation for the drastically differing convergence behavior?
</div>

In [ ]:
"""Visualization"""

A1 = np.diag([10,2,1])
A2 = np.diag([10,8,1])
A3 = np.diag([10,9.9,1])

AA = np.array([A1,A2,A3])

# for i in range(len(AA)):
#     plot_convergence(plt.gca(), power_iteration(AA[i])[0]) # 0 since we want to get back mu for every interation

for i in AA: # We get each element one after the other. (A1, ...)
    mue = power_iteration(i)[0] # 0 since we want to get back mu for every interation
    bb = power_iteration(i)[1] # 1 to get b
    plot_convergence(plt.gca(), mue)
    
    print( "largest eigenvalue =", mue[-1],"second largest eigenvalue =", mue[-2], "(%d iterations)"%len(mue) )
    print( "corresponding eigenvector = ",bb)

"""Explanation"""

# Power Iteration tends to converge the fastest, the greater the separations between the magnitudes of the Eigenvalues are.
# So convergence would depend on the ratio of the largest to the second largest Eigenvalues.
# In our case A2 and A3 would look the most similar to each other(allbeit not so much some times), than they would to A1.
# A1's first two largest Eigenvalues have a clear separation, so it tries to converge with as few iterations as possible, and stabilizes towards its largest eigenvalue.
# A1 can be identified as the blue line, and A2 and A3 would be the orange and green lines respectively.


<div class="alert alert-info">

### QR Decomposition using the Householder Transformations
</div>

In exercise 5, we have seen the QR decomposition of a matrix using the Gram-Schmidt process. In this exercise, the task is to implement it using the more stable Householder transformations. 

<div class="alert alert-success">

**Task 3**: Complete the function `QR_decomposition_Householder` below to implement the QR decomposition of the given square matrix `A`. The function should simply return the $Q$ and $R$ matrices as outputs. 
</div>

In [ ]:
"""
QR Decomposition with Python and NumPy, Quantstart, Stand 13.12.23, URL: https://www.quantstart.com/articles/QR-Decomposition-with-Python-and-NumPy/
Bei der def householder(A) haben wir geschaut, wie die Implementation mit Household aussehen kann,
jedoch haben wir unsere Implementation wie folgt angepasst.
Besonders Schwierig war die Einschätzung der range.
"""
# Zuerst Hilfsmethoden. Norm normalisiert für einen Vektor x, während mult_matrix nur 2 Matrizen der selben Dimension multipliziert.

def norm(x):
    return np.sqrt(sum([x_i**2 for x_i in x]))

def mult_matrix(M, N):                                                                  
    return np.dot(M,N)

def QR_decomposition_Householder(A):
    """
    Führt eine auf Householder-Reflexionen basierende QR-Zerlegung der                                               
    Matrix A. Die Funktion gibt Q, eine orthogonale Matrix und R, eine                                                  
    obere Dreiecksmatrix, so dass A = QR ist.
    """
    n = A.shape[0]
        
    Q = np.identity(n)    
    R = np.copy(A)
    I = np.identity(n)

    

    for j in range(n-1): # n * m Matrix! n = m v n != m
        for i in range(n-1):
            x = R[j:, j]  # Von der Spalte j n der Reihe j abwärts
            z = I[j:, j]  
        sign = -1 if x[0] < 0 else 1 if x[0] > 0 else 0 # definiert das Vorzeichen!
        x_normalized = np.multiply(sign, norm(x)) # Vorzeichen auf die Normalisierung von x anwenden

        v_1 = x + x_normalized * z # x + ||x|| * z
        v_1_normalized = norm(v_1)
        v = v_1 / v_1_normalized

        Q_min = np.identity(n - j) - 2.0 * np.outer(v, v) # Definition outer: outer product of two vectors. First input vector. Input is flattened if not already 1-dimensional.

        Q_t = np.identity(n) 
        Q_t[j:, j:] = Q_min # Für obere Dreiecksmatrix [j:, j:]
        
        Q = mult_matrix(Q, Q_t)
        R = mult_matrix(Q_t, R) 

    return Q,R

In below, we can check the correctness of the implementation on a test matrix using the identities $A = QR$ and $Q^\top Q = I$

In [ ]:
A = np.random.rand(4,4)
Q,R = QR_decomposition_Householder(A)
Acheck = Q@R
print(A-Acheck)
Icheck = np.transpose(Q)@Q
print(Icheck)

<div class="alert alert-info">

### Computing Eigenvalues using the QR Algorithm
</div>

<div class="alert alert-success">

**Task 4**: Complete the function `compute_eigenvalues`, which computes the eigenvalues of the square matrix $A$ using the QR algorithm. The function should return the vector of eigenvalues as the output. 
</div>

In [ ]:
def compute_eigenvalues(A):
    eigvals = np.random.random(A.shape[1])

    for i in range(100):
        if i%2 == 0:
            Q, R = QR_decomposition_Householder(A)
            A = Q@R
        else:
            A = R@Q

    eigvals = np.diag(A)

    return eigvals

Now we can check the implementation on a random matrix $A$, which has real eigenvalues. We can easily generate such a matrix using the diagonalization
\begin{equation}
A = S \Lambda S^{-1}
\end{equation}
We first generate a diagonal matrix 

In [ ]:
N = 6
Lambda = np.diag(np.random.rand(N)*10)
print(Lambda)


Now we generate a square random $S$ matrix. Note that this matrix will be invertible with almost certain probability. Using $S$ and $\Lambda$, we can generate a random matrix with real eigenvalues to test the implementation

In [ ]:
S = np.random.random((N,N))
Sinv = np.linalg.inv(S)
A = S@Lambda@Sinv
print(A)

The eigenvalues of $A$ can be easily found by using the `numpy.linalg.eigvals` function:

In [ ]:
eigenvaluesOfA = np.linalg.eigvals(A)
print(eigenvaluesOfA)

Now we are ready to use 'compute_eigenvalues'. If the implementation is correct, it should return exactly the same values as above

In [ ]:
eigenvaluesOfA = compute_eigenvalues(A)
print(eigenvaluesOfA)